In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch 
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
data_dir = '../input/animefacedataset'
print(len(os.listdir(data_dir + "/images")))

In [ ]:
image_size = 64
batch_size = 128
stats = (0.5,0.5,0.5) , (0.5,0.5,0.5)

In [ ]:
train_ds = torchvision.datasets.ImageFolder(data_dir , transform = T.Compose([T.Resize(image_size),
                                                                              T.CenterCrop(image_size),
                                                                              T.ToTensor(),
                                                                              T.Normalize(*stats)
                                                                             ]))

train_dl = torch.utils.data.DataLoader(train_ds ,batch_size, shuffle = True , num_workers = 3 , pin_memory = True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def to_device(data,device):
    if isinstance(data , (list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device)

class DeviceDataLoader():
    def __init__(self,dl,device):
        super().__init__()
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch,self.device)
            
    def __len__(self):
        return len(self.dl)

In [ ]:
device


In [ ]:
def denorm(img_tensors):
    return img_tensors*stats[1][0] + stats[0][0]

def show_image(images , nmax=64):
    fig , ax = plt.subplots(figsize=(8,8))
    ax.set_xticks([]); ax.set_yticks([]);
    ax.imshow(torchvision.utils.make_grid(denorm(images.detach()[:nmax]), nrow = 8).permute(1,2,0))
    
def show_batch(dl , nmax = 64):
    for img , _ in dl:
        show_image(img,nmax)
        break;

In [ ]:
show_batch(train_dl)

In [ ]:
train_dl = DeviceDataLoader(train_dl , device)

In [ ]:
discriminator = torch.nn.Sequential(
            
                torch.nn.Conv2d(3 , 64 , kernel_size=4, stride = 2, padding = 1 , bias = False),
                torch.nn.BatchNorm2d(64),
                torch.nn.LeakyReLU(0.2, inplace = True),
    
                torch.nn.Conv2d(64 , 128 , kernel_size = 4 , stride = 2 , padding = 1 , bias = False),
                torch.nn.BatchNorm2d(128),
                torch.nn.LeakyReLU(0.2, inplace=True),
    
                torch.nn.Conv2d(128 , 256 , kernel_size = 4 , stride = 2 , padding = 1 , bias = False),
                torch.nn.BatchNorm2d(256),
                torch.nn.LeakyReLU(0.2 , inplace = True),
    
                torch.nn.Conv2d(256 , 512 , kernel_size = 4 , stride = 2 , padding = 1 , bias = False),
                torch.nn.BatchNorm2d(512),
                torch.nn.LeakyReLU(0.2 , inplace = True),
    
                torch.nn.Conv2d(512 , 1 , kernel_size = 4 , stride = 1 , padding = 0 ,bias = False),
    
                torch.nn.Flatten(),
                torch.nn.Sigmoid()

                )

In [ ]:
latent_size = 128

In [ ]:
discriminator

In [ ]:
latent_size = 128

In [ ]:
generator = torch.nn.Sequential(
                # in: latent_size x 1 x 1
                
                torch.nn.ConvTranspose2d(latent_size , 512 , kernel_size = 4 , stride = 1 , padding = 0 , bias = False),
                torch.nn.BatchNorm2d(512),
                torch.nn.ReLU(True),
                # out: 512 x 4 x 4
    
                torch.nn.ConvTranspose2d(512 , 256 , kernel_size = 4 , stride=2 , padding=1 , bias = False),
                torch.nn.BatchNorm2d(256),
                torch.nn.ReLU(True),

                torch.nn.ConvTranspose2d(256 , 128 , kernel_size = 4 , stride = 2 , padding = 1 , bias = False),
                torch.nn.BatchNorm2d(128),
                torch.nn.ReLU(True),

                torch.nn.ConvTranspose2d(128 , 64 , kernel_size = 4 , stride = 2 ,padding = 1 , bias = False),
                torch.nn.BatchNorm2d(64),
                torch.nn.ReLU(True),
    
                torch.nn.ConvTranspose2d(64 , 3 , kernel_size = 4 , stride = 2 , padding = 1 , bias = False ),
                torch.nn.Tanh()
                # out: 3 x 64 x 64
    
)

In [ ]:
generator

In [ ]:
xb = torch.randn(batch_size,latent_size,1,1)
fake_images = generator(xb)
print(fake_images.shape)
show_image(fake_images)

In [ ]:
generator = to_device(generator,device)
discriminator = to_device(discriminator , device)

In [ ]:
def train_discriminator(real_images , opt_d):
    opt_d.zero_grad()
    
    real_preds = discriminator(real_images)
    real_targets = torch.ones(real_images.size(0) , 1 , device = device)
    real_loss = torch.nn.functional.binary_cross_entropy(real_preds , real_targets)
    real_score = torch.mean(real_preds).item()
    
    latent = torch.randn(batch_size , latent_size , 1,1 , device = device)
    fake_images = generator(latent)
    
    fake_preds = discriminator(fake_images)
    fake_targets = torch.zeros(fake_images.size(0) , 1 , device = device)
    fake_loss = torch.nn.functional.binary_cross_entropy(fake_preds , fake_targets)
    fake_score = torch.mean(fake_preds).item()
    
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score , fake_score

In [ ]:
def train_generator(opt_g):
    opt_g.zero_grad()
    
    latent = torch.randn(batch_size , latent_size , 1 , 1, device=device)
    fake_images = generator(latent)
    
    preds = discriminator(fake_images)
    targets = torch.ones(batch_size , 1 , device=device)
    loss = torch.nn.functional.binary_cross_entropy(preds, targets)
    
    loss.backward()
    opt_g.step()
    
    return loss.item()

In [ ]:
from torchvision.utils import save_image

sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

fixed_latent = torch.randn(64, latent_size, 1, 1, device=device)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def fit(epochs , lr , start_idx=1):
    torch.cuda.empty_cache()
    
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    opt_d = torch.optim.Adam(discriminator.parameters() , lr=lr , betas=(0.5 , 0.999))
    opt_g = torch.optim.Adam(generator.parameters() , lr=lr , betas=(0.5,0.999))
    
    for epoch in range(epochs):
        for real_images , _ in tqdm(train_dl):
            
            loss_d , real_score , fake_score = train_discriminator(real_images , opt_d)
            loss_g = train_generator(opt_g)
            
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # Save generated images
        save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
lr = 0.0002
epochs = 25

In [ ]:
history = fit(epochs,lr)

In [ ]:
print(discriminator(generator(fixed_latent)).shape)

In [ ]:
generator(fixed_latent).shape

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history

In [ ]:
torch.save(generator.state_dict() , 'G.ckpt')
torch.save(discriminator.state_dict() , 'D.ckpt')

In [ ]:
from IPython.display import Image

In [ ]:
Image('./generated/generated-images-0024.png')

In [ ]:
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');



In [ ]:
import cv2
import os

vid_fname = 'gans_training.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'generated' in f]
files.sort()

out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'MP4V'), 1, (530,530))
[out.write(cv2.imread(fname)) for fname in files]
out.release()